## Blackjack Game by Leonardo

In [1]:
from IPython.display import Image, display
import ipyplot
import random
from art import tprint
import cv2
import glob
import numpy as np
import time

## Global variable declarations:

In [2]:
game_break = False
round_number = 1

player_wins = 0
cpu_wins = 0
ties = 0

round_break = False
turn_number = 1
draw_or_stop = "DRAW"

cards = []
player_cards = []
cpu_cards = []

player_scores = []
cpu_scores = []

player_total_score = 0
cpu_total_score = 0

player_money = 0
cpu_money = 0

In [3]:
deck = {}
# The deck will be composed by a dictionary, with keys defined as the card name and the values defining
# the points/score for each card

for i in ["♠", "♥", "♦", "♣"]:
    for j in range(2,11):
        deck[str(j)+i] = j

    for k in ["J", "Q", "K"]:
        deck[str(k)+i] = 10

    deck["A"+i] = "choose"

cards = list(deck.keys())

## Functions declarations

In [4]:
def welcome_screen():
    tprint("BLACK",font="block")
    tprint("JACK",font="block")
    hello = '''Are you ready for'''
    for part in hello.split('\n'):
        print(f'{part.upper():^120}')
    return None

In [5]:
def reset_all(deck=deck):
    """This function has the objective of reseting:
    1. the deck of cards after every round, since the deck is used with no replacement of cards during each turn of play
    2. all the other variables related to the player and the CPU
    
    Input: deck (as a dictionary)
    Output: cards = multiple times the deck (as a list)"""
    
    global turn_number, round_break, game_break, draw_or_stop, cards, player_cards, player_scores, player_total_score, cpu_cards, cpu_scores, cpu_total_score
    
    turn_number = 1
    round_break = False
    draw_or_stop = "DRAW"
    
    cards = []
    
    player_cards = []
    player_scores = []
    player_total_score = 0
    player_money = 0
    
    cpu_cards = []
    cpu_scores = []
    cpu_total_score = 0

    
    number_of_decks_used = input("Choose the number of decks you would like to play with (from 1 to 8): \n")
    
    while number_of_decks_used not in range(1,9):
        try:
            number_of_decks_used = int(number_of_decks_used)
        except:
            number_of_decks_used = input("Please, try again. You need to choose an integer between 1 and 8 for the number of decks:\n")
        if number_of_decks_used not in range(1,9):
            number_of_decks_used = input("Please, try again. You need to choose an integer between 1 and 8 for the number of decks:\n")
    
    cards = []
    for i in range(number_of_decks_used):
        cards += list(deck.keys())
    
    print(f"\nNice choice! Your deck has then {len(cards)} CARDS and here are the cards in your full deck:")
    print(cards)
    
    time.sleep(1)
    
    option_quit_game()
    
    return None

In [6]:
def bets():
    if player_money == 0 & cpu_money == 0:
        cpu_money = player_money = input("Place the money that you want to place to your pot: ")
    
    print(f"You have {player_money}$ available for betting")
    
    flag == True
    
    while player_bet != 0 and cpu_bet != 0 and flag == True:
        try:
            bet = int(input(f"Place your bet for the round {round_number} and turn {turn_number} : "))
            if int(bet) != bet:
                Flag == True
            else:
                flag == False
        except:
            print("You should input a number for the bet. Please, try again.")
    
    return None

In [7]:
def choice_of_cards(cards=cards):
    """
    Function randomly choosing a card for the player or the CPU.
    """
    chosen_card = random.choices(cards)
    return chosen_card[0]

In [8]:
def option_quit_game():
    """
    Function to give the player the option to continue the game or to quit in between every new card been handled.
    """
    global round_break, game_break
    
    check = input("\nPress Enter to continue... Or Q to quit.")
    if check == 'Q':
        round_break = True
        game_break = True
    print()
    return None

In [9]:
def card_points(card_chosen, deck=deck):
    """This function determines the punctuation of each card (either draw by the player or the cpu)
    
    Input: card to be evaluated and the deck dictionary
    Output: punctuation of the card
    """
    
    #print(deck[card_chosen])
    
    if deck[card_chosen] in range (0,11):
        return deck[card_chosen]
    
    elif deck[card_chosen] == "choose":
        flag = True
        points = input("Nice ! You got an Ace ! Choose if you want to attribute 1 or 11 points to this card :\n")
        print()
        
        while flag:
            try:
                points = int(points)
            except:
                points=input("Please, try again. You need to choose 1 or 11 points to this card :\n")
            else:
                if int(points) == 1 or int(points) == 11:
                    flag = False
                    break
                else:
                    points=input("Please, try again. You need to choose 1 or 11 points to this card :\n")
        return int(points)
    
    else:
        print("There's a problem with the ponctuation of cards.\n")
        return 0

In [10]:
def player_choice():
    global draw_or_stop
    
    if draw_or_stop == "STOP":
        return "STOP"
    else:
        draw_or_stop = input("Type DRAW to draw a new card or STOP if you want to STOP drawing cards:")
        
        while draw_or_stop.lower() not in ["draw", "stop"]:
            draw_or_stop = input("Try again. Type DRAW to draw a new card or STOP if you want to STOP drawing cards:")
        
        draw_or_stop = draw_or_stop.upper()
        
        return None

In [11]:
#from IPython.display import Image
#Image(filename='9♥.jpg', width = 100)

def show_cards(cards):
    
    cards_images = []
    files = glob.glob("/Users/oleoaraujo/Documents/Github/data-ft-par-labs/Projects/Week-1/your-project/*.jpg")

    for card in cards:
        for myFile in files:
            if myFile == "/Users/oleoaraujo/Documents/Github/data-ft-par-labs/Projects/Week-1/your-project/"+card+".jpg":
                image = cv2.imread(myFile)
                cards_images.append(image)
    
    ipyplot.plot_images(cards_images, max_images=20, img_width=100)
    
    return None

In [12]:
def player_turn(turn_number, cards=cards, deck=deck):
    """
    This function defines all the tasks that are executed when a card is to be drawn by the player,
    updating the necessary variables : from player_card, to the new cards (deck), and the player_score.
    """
    
    global player_total_score, player_scores, player_cards, draw_or_stop
    
    print(f"------------------------------------ TURN {turn_number} ------------------------------------")
    
    print("**PLAYER TURN:**")
    player_card = None
    
    if turn_number <= 2:
        print(f"For this a new card has automatically been drawn to the player by the dealer (CPU).")
        draw_or_stop = "DRAW"
    
    elif turn_number >2:
        player_choice()
            
        
    if draw_or_stop == "DRAW":
        player_card = choice_of_cards(cards)
        print(f"Here's the card : {player_card}.\n")
        
        # Drawing a card from the deck: removing it from the deck and allocating it to the player
        cards = cards.remove(player_card)
        player_cards.append(player_card)
        
        # Showing all the cards that have been drawn up to the player up until now
        show_cards(player_cards)
        
        # Score updates
        player_scores.append(card_points(player_card))
        player_total_score = sum(score for score in player_scores)
        
        #Image(filename=player_card+'.jpg', width=100)
    
    return None

In [13]:
def cpu_turn(turn_number=turn_number, cards=cards, deck=deck):
    
    global cpu_total_score, cpu_cards, round_break, game_break
    
    # Drawing a card from the deck: removing it from the deck and allocating it to the cpu:
    if turn_number<=2 or cpu_total_score < 16:
        cpu_card = choice_of_cards(cards)
        
        print("**CPU TURN:**")
        print("A new card has been drawn to the CPU.")
        print(f"Here's the card : {cpu_card}\n")
        
        cards = cards.remove(cpu_card)
        cpu_cards.append(cpu_card)
        
        # Showing all the cards that have been drawn up to the player up until now
        show_cards(cpu_cards)
        
        # Score updates
        cpu_scores.append(card_points(cpu_card))
        cpu_total_score = sum(score for score in cpu_scores)
    
    else:
        print(f"It would be the CPU turn, but since it already reached the score of {cpu_total_score}, it has finished playing during this round.\n")
    
    option_quit_game()
    
    return None

In [14]:
def print_turn():
    
    if game_break == True:
        return None
    
    print(f"\n----------------------------------------------")
    print(f"-------------- TURN {turn_number} - RESULTS --------------")
    print(f"----------------------------------------------")
    print("")
    print("The PLAYER cards are:", player_cards)
    print("The PLAYER scores are:", player_scores)
    print("The PLAYER total score is:", player_total_score)
    print("")
    print("The CPU cards are:", cpu_cards)
    print("The CPU scores are:", cpu_scores)
    print("The CPU total score is:", cpu_total_score)
    print("")
    print(f"----------------------------------------------")
    print(f"----------------------------------------------")
    print(f"----------------------------------------------")
    
    option_quit_game()
    
    return None

In [15]:
def print_round_winner():
    
    global player_wins, cpu_wins, ties
    
    if player_total_score > cpu_total_score and player_total_score <= 21:
        player_wins += 1
        print("PLAYER is the winner of this round!")
        print()
    
    elif cpu_total_score > 21 and player_total_score <= 21:
        player_wins += 1
        print("PLAYER is the winner of this round!")
        print()
    
    elif cpu_total_score > player_total_score and cpu_total_score <= 21:
        cpu_wins += 1
        print("CPU is the winner of this round!")
        print()
        
    elif player_total_score > 21 and cpu_total_score <= 21:
        cpu_wins += 1
        print("CPU is the winner of this round!")
        print()
    
    else:
        ties += 1
        print("It's a tie !")
        print()
        
    print(f"The score of the game in the round {round_number} is:")
    print(f"PLAYER : {player_wins}")
    print(f"CPU : {cpu_wins}")
    print(f"ties : {ties}")
    print()
    
    option_quit_game()
    
    return None

In [16]:
def print_final_result():
    
    print(f"---------------------------------------------")
    print(f"---------------------------------------------")
    
    if player_wins > cpu_wins:
        print("\nPLAYER is the absolute winner!\n")
    if cpu_wins > player_wins:
        print("\nCPU is the absolute winner!\n")
    else:
        print("\nThere's no winner for this game!\n")
    
    print(f"---------------------------------------------")
    print(f"---------------------------------------------")

    return None

## Main Code Execution

In [ ]:
# Looping each ROUND of play
welcome_screen()
while game_break == False:
    
    # Reseting the main variables
    reset_all()
    
    # Looping each TURN of play
    while round_break == False:
        
        player_turn(turn_number)
        time.sleep(1)
        cpu_turn(turn_number)
        
        #check_winner()
        
        if turn_number>2 and cpu_total_score >=16 and draw_or_stop.casefold() == "STOP":   
            round_break = True
        elif cpu_total_score>21 or player_total_score>21:
            round_break = True
        
        if len(cards) > 0 and game_break == False:
            print_turn()
        
        turn_number += 1
        
    if len(cards) >0 and game_break == False:
        print_round_winner()
    
    round_number += 1
    
print_final_result()


 .----------------.  .----------------.  .----------------.  .----------------.  .----------------.
| .--------------. || .--------------. || .--------------. || .--------------. || .--------------. |
| |   ______     | || |   _____      | || |      __      | || |     ______   | || |  ___  ____   | |
| |  |_   _ \    | || |  |_   _|     | || |     /  \     | || |   .' ___  |  | || | |_  ||_  _|  | |
| |    | |_) |   | || |    | |       | || |    / /\ \    | || |  / .'   \_|  | || |   | |_/ /    | |
| |    |  __'.   | || |    | |   _   | || |   / ____ \   | || |  | |         | || |   |  __'.    | |
| |   _| |__) |  | || |   _| |__/ |  | || | _/ /    \ \_ | || |  \ `.___.'\  | || |  _| |  \ \_  | |
| |  |_______/   | || |  |________|  | || ||____|  |____|| || |   `._____.'  | || | |____||____| | |
| |              | || |              | || |              | || |              | || |              | |
| '--------------' || '--------------' || '--------------' || '--------------' || '--------